How to start a webdriver client in node?

In [ ]:
var importer = require('../Core');
var sync = require('wdio-sync');
var MAX_SESSIONS = 1;
//var MAX_SESSIONS = 36;

var client;
function createWebdriverClient(host, port) {    
    var webdriverServer = {
        services: ['selenium-standalone', 'chromedriver'],
        sync: false,
        debug: false,
        host: host || 'localhost',
        port: port || 4444,
        logLevel: 'command',
        baseUrl: 'https://webdriver.io',
        pageLoadStrategy: 'eager',
        connectionRetryTimeout: 1000,
        desiredCapabilities: {
            browserName: 'chrome',
            chromeOptions: {
                prefs: {
                    'download.default_directory': '/data/downloads',
                    'profile.default_content_setting_values.notifications': 2,
                    'exited_cleanly': true,
                    'exit_type': 'None'
                },
                args: [
                    // TODO: https://superuser.com/questions/461035/disable-google-chrome-session-restore-functionality
                    'user-data-dir=/tmp/profile-' + MAX_SESSIONS,
                    // 'start-fullscreen',
                    'no-sandbox',
                    'disable-session-crashed-bubble',
                    'disable-infobars',
                    'new-window',
                    'disable-geolocation',
                    'disable-notifications',
                    'show-saved-copy',
                    'silent-debugger-extension-api'
                    //'kiosk'
                ]
            }
        },
    };
    
    client = require('webdriverio').remote(webdriverServer);
    client.on('error', e => console.log(e.message));
    client.on('end', () => console.log('Daemon: Closing browser'));
    const connectSession = importer.import('connect webdriver session', {client});
    return connectSession();
};
module.exports = createWebdriverClient;


connect to webdriver session?

find webdriver sessions?


In [ ]:
var importer = require('../Core');
var readSessions = importer.import('load webdriver sessions');
var {
    getSessions,
    lockPromise
} = importer.import('manage webdriver sessions', {client});

var TIMEOUT = 10000;
var MAX_SESSIONS = 1;

function connectSession() {
    return client
        .then(() => lockPromise(true, true))
        .getSessions(true)
        // save current session
        .then(validSessions => {
            var sessions = readSessions();
            // the next null or end will be the next available profile id
            var index = sessions.map(s => s[1]).indexOf(validSessions[0] || 0);
            if(index === -1) {
                index = sessions.length;
            }
            if(index >= MAX_SESSIONS) {
                throw new Error('Already running max sessions ' + MAX_SESSIONS);
            }
            client.options.connectionRetryTimeout = TIMEOUT;
            client.options.desiredCapabilities.chromeOptions.args[0] = 'user-data-dir=/tmp/profile-' + index;
            // TODO: fix this, doesn't work on second init, keeps opening new windows if chrome profile path is alreading open for read/write
            if(typeof validSessions[0] !== 'undefined') {
                console.log('using existing session ' + index + ' - ' + validSessions[0]);
                client.requestHandler.sessionID = validSessions[0];
                return client.session()
            } else {
                console.log('new session ' + index);
                return client.init().session();
            }
        })
        .catch(e => {
            client.requestHandler.sessionID = null;
            console.log(e);
        })
        .then(() => lockPromise(false, true))
}
module.exports = connectSession;


Load webdriver sessions?


In [ ]:
var fs = require('fs');
var path = require('path');

var TOKEN_DIR = path.join(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE, '.credentials');
var SESSIONS_PATH = path.join(TOKEN_DIR, 'sessions.json');

var sessions = {};
var sessionModified = 0;

function readSessions() {
    try {
        if(fs.existsSync(SESSIONS_PATH)
           && fs.statSync(SESSIONS_PATH).mtime.getTime() > sessionModified) {
            sessionModified = fs.statSync(SESSIONS_PATH).mtime.getTime();
            sessions = JSON.parse(fs.readFileSync(SESSIONS_PATH)
                .toString());
        }
    } catch (e) {
        sessions = [];
    }
    return sessions;
};
module.exports = readSessions;


update session?


In [ ]:
var lockFile = require('lockfile');
var fs = require('fs');
var path = require('path');
var importer = require('../Core');
var readSessions = importer.import('load webdriver sessions');

var TOKEN_DIR = path.join(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE, '.credentials');
var SESSIONS_PATH = path.join(TOKEN_DIR, 'sessions.json');
var INIT_WAIT = 60000; // 36 * session test time * number of simultaneous sessions
var UPDATE_WAIT = 1000;

// lock / unlock
// insert - posibility of a session being reused, but sych session starts
function lockPromise(lock = true, init = false) {
    console.log((init ? 'init' : 'update') + ' - ' + (lock ? 'locking' : 'unlocking'));
    return new Promise((resolve, reject) => {
        const func = lock ? lockFile.lock : lockFile.unlock;
        const p = SESSIONS_PATH + '.' + (init ? 'init' : 'update') + '.lock';
        return func.apply(lockFile, [p].concat(lock ? [{
            stale: init ? INIT_WAIT : UPDATE_WAIT,
            wait: init ? INIT_WAIT : UPDATE_WAIT
        }] : []).concat([(err) => {
            if(err) {
                return reject(err);
            }
            console.log((init ? 'init' : 'update') + ' - ' + (lock ? 'lock' : 'unlock'));
            resolve();
        }]));
    });
}

function updateOrAddSession(currentSession) {
    const sessions = readSessions();
    if(!currentSession) {
        return sessions;
    }
    // don't update sessions while scanning
    const updateSession = sessions.filter(s => s[1] === currentSession)[0];
    if(typeof updateSession !== 'undefined') {
        console.log('update ' + currentSession);
        updateSession[0] = (new Date()).getTime();
    } else {
        console.log('insert ' + currentSession);
        const oldSession = sessions[sessions.length] = [];
        // http://www.english.upenn.edu/~jenglish/English104/tzara.html
        oldSession[1] = currentSession;
        oldSession[0] = (new Date()).getTime();
    }
    fs.writeFileSync(
        SESSIONS_PATH,
        JSON.stringify(sessions, null, 4));
    return sessions;
}
module.exports = {
    updateOrAddSession,
    lockPromise
};


Manage webdriver sessions?



In [ ]:
var importer = require('../Core');
var readSessions = importer.import('load webdriver sessions', {client});
var {
    verifySession,
    lockPromise
} = importer.import('verify session', {client});

var TIMEOUT = 10000;

function getSessions(inactive = false) {
    const sessions = readSessions();
    const original = client.requestHandler.sessionID;
    var active = [].concat(sessions)
        .filter(session => typeof session[1] !== 'undefined'
                && session[1] !== null && session.length > 0);
    if(inactive) {
        active = active.filter(session => (new Date()).getTime() - session[0] > TIMEOUT);
    }
    var cancelled = false;
    return importer.runAllPromises(active.map(session => (resolve) => {
        if(cancelled) {
            return resolve();
        }
        return verifySession(session)
            .catch(e => console.log(e))
            .then(r => {
                // only try to find 1 decent session
                if(inactive && typeof r !== 'undefined') {
                    cancelled = true;
                }
                return resolve(r);
            })
    }))
        .then(available => {
            client.requestHandler.sessionID = original;
            return available
                .filter(sess => typeof sess !== 'undefined' && sess !== null)
                .filter((elem, pos, arr) => arr.indexOf(elem) === pos)
        })
}

if(typeof client.getSessions === 'undefined') {
    client.addCommand('getSessions', getSessions);
}

module.exports = {
    getSessions,
    lockPromise
};



verify session?



In [ ]:
var importer = require('../Core');
var {
    updateOrAddSession,
    lockPromise
} = importer.import('update session');

var scanning = false;
var TIMEOUT = 10000;

function verifySession(session) {
    client.requestHandler.sessionID = session[1];
    return client
        .then(() => scanning = true)
        .windowHandle()
        .then(r => client.window(r.value))
        .session()
        .then(s => s.sessionId)
        .catch(e => {
            if(e.message === 'ESOCKETTIMEDOUT' || e.message.indexOf('no such session')) {
                 session[1] = '';
            } else {
                console.log('error ' + session[1]);
                console.log(e)
            }
            // if the session is really old and has an error delete it from the list
            //const index = sessions.map(s => s[1]).indexOf(session[1]);
            //sessions[index][1] = null;
        })
        .then(r => {
            scanning = false;
            return r;
        })
}
module.exports = {
    lockPromise, verifySession
};

var sessions = [];
if(typeof client.verifySession === 'undefined') {
    client.addCommand('verifySession', verifySession);
    client.on('result', (result) => {
        if(scanning) {
            return;
        }
        const currentSession = client.requestHandler.sessionID;
        const updateSession = sessions.filter(s => s[1] === currentSession)[0];
        if(typeof updateSession !== 'undefined') {
            if((new Date()).getTime() - updateSession[0] <= TIMEOUT / 2) {
                return;
            }
        }

        return lockPromise(true)
            .then(() => sessions = updateOrAddSession(currentSession))
            .then(() => lockPromise(false))
            .catch(e => console.log(e));
    });
}


In [ ]:
$$.async();
var client = createWebdriverClient('localhost', 4444)
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e));


Initializing webdriver on localhost


In [ ]:
$$.async();
client.windowHandles()
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e));


How to end the webdriver service?

In [ ]:
client.endAll();



TODO: add decorated logging with screenshots of buttons results can be used: https://github.com/megamindbrian/bots/blob/master/bots/server.js

TODO: transfer state and cache to client

